# **RAG Implementation**

On va commencer par installer les bibliothèques nécessaires

In [1]:
!pip install langchain_community huggingface-hub transformers accelerate bitsandbytes pymupdf langchain faiss-cpu sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 105.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.

## **Initiation du LLM utilisé**

Insérez votre token accessible sur le site [Hugging Face](https://huggingface.co/settings/tokens)

In [2]:
from huggingface_hub import notebook_login

notebook_login()

J'ai utilisé le modèle **bigscience/bloomz-7b1-mt**, vu qu'il est adapté au langage français, et qu'il ne nécessite pas beacoup de stockage.

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "bigscience/bloomz-7b1-mt"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/14.1G [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# Test de génération
prompt = ' c est quoi le nom de Einsten ?'
response = generator(prompt, max_new_tokens=400,  do_sample=True, temperature=0.1)
print(response[0]["generated_text"])

 c est quoi le nom de Einsten ? Albert


## **Files Upload**

Dans cette section on va uploader nos fichiers et construire les chunks et les indexer dans notre vectorstore.

In [10]:
import fitz

def extract_text_by_page(file_path):
    doc = fitz.open(file_path)
    pages = []
    for i, page in enumerate(doc):
        text = page.get_text()
        if text.strip():  # ignorer les pages vides
            pages.append({"page": i + 1, "text": text})
    return pages

In [11]:
text_HG = extract_text_by_page('/content/CollinsSuzanne-HungerGames-1HungerGames2008.French.ebook_1.pdf')

In [12]:
text_HP1 = extract_text_by_page('/content/harry-potter-1-lecole-des-sorciers.pdf')

In [13]:
text_HP2 = extract_text_by_page('/content/harry-potter-2-la-chambre-des-secrets.pdf')

## **Construction du vectorstore**

In [14]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

On implémente maintenant la fonction qui va créer nos chunks, il  contribue en même temps le nom du roman et le numéro de la page du chunk en tant que metadata.

In [18]:
def split_pages_to_chunks(pages, book_title):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    all_chunks = []
    for page in pages:
        sub_docs = splitter.create_documents([page["text"]])
        for doc in sub_docs:
            doc.metadata["source"] = f'From {book_title} book, page {page["page"]}'

        all_chunks.extend(sub_docs)
    return all_chunks

In [19]:
chunks = split_pages_to_chunks(text_HG, 'Hunger Games') + split_pages_to_chunks(text_HP1, 'Harry Potter, Book 1') + split_pages_to_chunks(text_HP2, 'Harry Potter, Book 2')

In [20]:
print(chunks[0])

page_content='Suzanne Collins
 
 
 
 
 
 
 
Hunger Games
Traduit de l’anglais (États-Unis)
par Guillaume Fournier
 
 
 
 
POCKET' metadata={'source': 'From Hunger Games book, page 2'}


On peut maintenant construire notre vectorstore en utilisant l'indexation Faiss.

In [21]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

<ipython-input-21-ee273c448272>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## **Notre système RAG**

La dernière étape est de regrouper tout ce qui a precédé pour construire notre RAG.

In [30]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

llm = HuggingFacePipeline(pipeline=generator)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)



## **Questions tests**

Notre système est prêt maintenant à répondre aux questions des utilisateurs.

In [28]:
def ask_question(query):
  result = qa_chain.run(query)
  print(result)

In [31]:
query = "Pourquoi Hermione s'est enfermée dans les toilettes des filles ? "
ask_question(query)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

les autres spectres.
—Oh non, dit Hermione en s'immobilisant. Vite, demi-tour, je ne veux pas parler à Mimi Geignarde...
—Qui ça ? dit Harry tandis qu'ils revenaient précipitamment sur leurs pas.
—Elle hante les toilettes des filles, au deuxième étage, dit Hermione.
—Les toilettes ?
—Oui. Elles ont été inutilisables pendant toute l'année parce qu'elle n'arrêtait pas de piquer des crises en

venait. Quelqu'un l'a essuyée.
—Elle était à peu près là, dit Ron qui avait retrouvé son sang-froid. Devant cette porte.
Il montra l'endroit, à quelques pas de la chaise de Rusard.
Ron tendit la main vers la poignée de la porte, mais il interrompit son geste.
—On ne peut pas entrer, dit-il. Ce sont les toilettes des filles.
—Il n'y a personne, dit Hermione en s'approchant à son tour. C'est là qu'habité Mimi Geignarde. Venez, on
va jeter u

In [33]:
query = "À quelle heure le train Poudlard Express est parti de la gare ? "
ask_question(query)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

dois prendre le train à la gare de King's Cross à onze heures, sur la voie 9 ¾, ajouta-t-il en regardant
le billet que Hagrid lui avait donné.

que je suis tout petit... Ah, on est arrivés.
Ils étaient devant la gare et il y avait un train pour Londres cinq minutes plus tard. Hagrid, qui ne

Ils arrivèrent devant King's Cross à onze heures moins le quart. Mr Weasley se précipita pour aller
chercher des chariots à bagages et ils s'engouffrèrent à grands pas dans la gare.
Harry avait déjà pris le Poudlard Express l'année précédente. La difficulté consistait à trouver la voie 9
3/4 qui n'était pas visible aux yeux des Moldus. Pour y accéder, il fallait traverser la barrière qui se
dressait entre les voies 9 et 10. C'était indolore mais on devait faire attention que les Moldus ne

avait retrouvé toute sa joie de vivre.
Bientôt, 

In [34]:
query = "Qui est Firenze et comment a-t-il aidé Harry ? "
ask_question(query)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Même s'il faut pour cela venir en aide à un humain.
Firenze partit alors au galop et Harry essaya de s'accrocher de son mieux tandis qu'ils plongeaient
dans la forêt, laissant Ronan et Bane derrière eux.
Harry n'avait aucune idée de ce qui se passait.
—Pourquoi Bane est-il tellement en colère ? demanda-t-il. C'est à cause de cette chose dont tu m'as
sauvé ?
Firenze ralentit l'allure et conseilla à Harry de baisser la tête pour ne pas se heurter aux branches

Harry, qui avait du mal à suivre les grandes enjambées du géant, lui raconta la visite de Dobby et ce que
les Dursley lui avaient fait subir.
—Maudits Moldus, maugréa Hagrid. Si j'avais su...
—Harry ! Harry ! Par ici !
Harry leva la tête et vit Hermione Granger en haut des marches qui menaient à l'entrée de Gringotts. Elle
se précipita à sa rencontre, ses cheveux bruns e

In [35]:
query = "Comment s'appelle l'ami roux de Harry ? "
ask_question(query)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

que Harry connaissait de vue, mais à qui il n'avait jamais parlé.
—Je m'appelle Justin Finch-Fletchley, dit le garçon d'une voix claironnante en serrant la main de Harry.
Je sais qui tu es, bien sûr, le célèbre Harry Potter... et toi, tu es Hermione Granger, toujours la meilleure

Potter...
Il sortit de sa poche la baguette magique de Harry et écrivit dans l'air en lettres scintillantes :
TOM ELVIS JEDUSOR
Puis il fit un mouvement avec la baguette et les lettres de son nom s'assemblèrent dans un ordre différent.
A présent, on pouvait lire :
JE SUIS VOLDEMORT
—Tu vois ? murmura-t-il. C'est un nom que j'utilisais déjà à Poudlard, pour mes amis les plus proches. Tu
crois donc que j'allais accepter le «jeu du sort » qui m'avait donné ce nom immonde de « Jedusor », légué

l'amitié..
—Oh, Harry, fais bien attention...
—Bois la pre

In [36]:
query = "Quel est le nom de la sœur de Katniss Everdeen dans Hunger Games ?  "
ask_question(query)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Suzanne Collins
 
 
 
 
 
 
 
Hunger Games
Traduit de l’anglais (États-Unis)
par Guillaume Fournier
 
 
 
 
POCKET

jusqu’au podium et lance son traditionnel :
— Joyeux Hunger Games ! Et puisse le sort vous être favorable ! Ses
cheveux roses sont sûrement une perruque, car ses boucles sont légèrement
de travers depuis qu’Haymitch a essayé de la prendre dans ses bras. Elle
s’étend un peu sur la fierté qu’elle éprouve à se trouver là, même si tout le
monde sait bien qu’elle n’espère qu’une chose, être promue dans un meilleur
district, avec des vainqueurs dignes de ce nom et non pas des ivrognes qui

morts tous les deux si elle n’avait pas mangé ces baies la première. (Il se
reprend.) Non, bien sûr que non. Tu les as reconnues tout de suite, pas vrai ?
Je fais oui de la tête.
— On appelle ça du sureau mortel.
— Rien que le nom 

In [37]:
query = "Quel est le point commun entre le père d'Harry Potter et le père du personnage de Hunger Games ? "
ask_question(query)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

tuer Lily et James Potter ? C'est une insulte ! Un scandale ! Harry Potter ne connaît même pas sa
propre histoire, alors que dans notre monde, tous les enfants connaissent son nom !
—Mais pourquoi ? Qu'est-ce qui s'est passé ? demanda Harry, avide de savoir.
La colère disparut du visage de Hagrid. Il eut soudain l'air très mal à l'aise.
Je ne m'attendais vraiment pas à ça, dit-il d'une voix inquiète. Quand Dumbledore m'a prévenu qu'il ne

—et Harry disait souvent qu'il avait l'air d'un cochon avec une perruque.
Harry essaya de disposer sur la table les assiettes remplies d'oeufs au bacon, ce qui n'était pas facile
en raison du peu de place qui restait. Pendant ce temps, Dudley comptait ses cadeaux. Lorsqu'il eut
terminé, ses joues s'affaissèrent.
—Trente-six, dit-il en levant les yeux vers ses parents. Ça fait deux de moins 

In [38]:
query = "Qui est le plus âgé des personnages principaux entre Harry Potter et Katniss Everdeen ? "
ask_question(query)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

—Moon... Nott... Le professeur McGonagall appela les noms qui commençaient par « P ». Parkinson...
les jumelles Patil... Perks, Sally-Anne... et, enfin...
—Harry Potter !
Lorsque Harry sortit du rang, des murmures s'élevèrent dans toute la salle.
—Elle a bien dit Potter ?
—Le Harry Potter ?
Avant que le chapeau lui tombe devant les yeux en le plongeant dans le noir absolu, Harry eut le
temps de voir les têtes qui se tendaient pour mieux le regarder.

étaient toujours un peu lents à la détente.
—Saint Potter, l'ami des Sang-de-Bourbe, dit lentement Malefoy. Encore un qui ne se conduit pas comme
un vrai sorcier, sinon, il ne se traînerait pas tout le temps avec cette parvenue d'Hermione Granger. Une
vraie Sang-de-Bourbe, celle-là. Quand on pense qu'il y a des gens qui considèrent Potter comme l'héritier
de Serpentard !
Harry e